# Etude de la stabilité du modèle dans le temps

On cherche la date pour laquelle on a à peu près la moitié du nombre total de clients (i.e. environ 96000) --> 48000 clients

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datetime import datetime

from package.outils import import_data
from sqlite3 import connect

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from math import pi
from sqlite3 import connect
from datetime import datetime

from evidently import Report
from evidently.presets import DataDriftPreset

from sklearn.preprocessing import PowerTransformer, StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans

In [2]:
date_fin = "'2018-01-15'"

sql_query = f"""
     WITH favourite_product AS (
     SELECT customer_id,
               product_category_name_english AS type_produit_prefere
          FROM (
                    SELECT o.customer_id,
                         p.product_category_name,
                         t.product_category_name_english,
                         count(p.product_id) AS nb_products_by_category,
                         ROW_NUMBER() OVER (PARTITION BY o.customer_id ORDER BY COUNT(p.product_id) DESC) AS rang
                    FROM orders o
                         JOIN
                         order_items i ON i.order_id = o.order_id
                         JOIN
                         products p ON p.product_id = i.product_id
                         JOIN
                         translation t ON p.product_category_name = t.product_category_name
                    GROUP BY o.customer_id,
                              p.product_category_name,
                              t.product_category_name_english
               )
          WHERE rang = 1
     ),

     -- CTE pour agréger les paiements par commande
     paiement_par_commande AS (
     SELECT order_id,
               SUM(payment_value) AS depense_commande
          FROM order_pymts
          GROUP BY order_id
     ),

     -- CTE pour agréger les items par commande (éviter la multiplication)
     items_par_commande AS (
     SELECT order_id,
               COUNT(order_item_id) AS nb_produits_commande,
               SUM(price) AS prix_produit_commande,
               SUM(freight_value) AS prix_transport_commande
          FROM order_items
          GROUP BY order_id
     ),

     -- CTE pour la moyenne des reviews par commande
     reviews_par_commande AS (
     SELECT order_id,
               AVG(review_score) AS score_moyen_commande
          FROM order_reviews
          GROUP BY order_id
     )

     -- Requête principale
     SELECT c.customer_id,
          c.customer_state,
          COUNT(DISTINCT o.order_id) AS nb_commandes,
          SUM(i.nb_produits_commande) AS nb_produits,
          SUM(i.prix_produit_commande) AS prix_produit_total,
          SUM(i.prix_transport_commande) AS prix_transport_total,
          SUM(p.depense_commande) AS depense_totale,
          (julianday({date_fin})-julianday(MAX(o.order_purchase_timestamp))) as recence,
          AVG(r.score_moyen_commande) AS score_moyen,
          f.type_produit_prefere
     FROM customers c
          JOIN
          orders o ON c.customer_id = o.customer_id
          LEFT JOIN
          items_par_commande i ON i.order_id = o.order_id
          LEFT JOIN
          reviews_par_commande r ON o.order_id = r.order_id
          LEFT JOIN
          paiement_par_commande p ON o.order_id = p.order_id
          LEFT JOIN
          favourite_product f ON f.customer_id = c.customer_id
     GROUP BY c.customer_unique_id;
"""

conn = connect(database="olist.db")
data = pd.read_sql(sql_query, conn)
print(data.shape)
print(data.dtypes)
data.head()

(96096, 10)
customer_id              object
customer_state           object
nb_commandes              int64
nb_produits             float64
prix_produit_total      float64
prix_transport_total    float64
depense_totale          float64
recence                 float64
score_moyen             float64
type_produit_prefere     object
dtype: object


,customer_id,customer_state,nb_commandes,nb_produits,prix_produit_total,prix_transport_total,depense_totale,recence,score_moyen,type_produit_prefere
0,fadbb3709178fc513abc1b2670aa1ad2,SP,1,1.0,129.90,12.00,141.90,-115.455868,5.0,bed_bath_table
1,4cb282e167ae9234755102258dd52ee8,SP,1,1.0,18.90,8.29,27.19,-112.466285,4.0,health_beauty
2,9b3932a6253894a02c1df9d19004239f,SC,1,1.0,69.00,17.22,86.22,310.121493,3.0,stationery
3,914991f0c02ef0843c0e7010c819d642,PA,1,1.0,25.99,17.63,43.62,94.146053,4.0,telephony
4,47227568b10f5f58a524a75507e6992c,SP,1,1.0,180.00,16.89,196.89,61.176597,5.0,telephony


Enlever les clients qui ont plus d'un pourcent d'erreur sur leur dépense totale. 

In [3]:
numerics = ["int16", "int32", "int64", "float16", "float32", "float64"]

In [4]:
print(f"Il y a {len(data)} clients avant filtrage des erreurs")
data_filtered = data.loc[
    np.abs(
        data["depense_totale"]
        - (data["prix_produit_total"] + data["prix_transport_total"])
    )
    < (0.01 * data["depense_totale"]),
    :,
]
data_filtered.dropna(inplace=True)
data_filtered = data_filtered.drop(
    ["prix_transport_total", "prix_produit_total"], axis=1
)
print(f"Il y a {len(data_filtered)} clients après filtrage des erreurs")
numeric_df = data_filtered.select_dtypes(include=numerics)
print(len(numeric_df.columns))

Il y a 96096 clients avant filtrage des erreurs
Il y a 93029 clients après filtrage des erreurs
5


Créer une pipeline avec le power transformer et le kmeans et faire un fit_transform de la pipeline sur le dataset.

In [5]:
from sklearn.pipeline import Pipeline

initial_model = Pipeline(
    steps=[
        ("scaler", PowerTransformer(method="yeo-johnson")),
        ("clusterer", KMeans(n_clusters=3, random_state=42)),
    ]
)
initial_model.fit(numeric_df)

Pipeline(steps=[('scaler', PowerTransformer()),
                ('clusterer', KMeans(n_clusters=3, random_state=42))])

Se déplacer d'une semaine dans le temps et prédire avec un predict à laide du kmeans pré entrainé. Faire une boucle sur le déplacement d'une semaine jusqu'à ce que le dataframe ait sa taille totale et faire à chaque fois, d'une part, la prédiction avec le modèle entrainé à la date initiale et un nouveau modèle entrainé à cette date. 

In [6]:
# Générer toutes les dates sur deux mois avec un pas d'une semaine :
initial_date = pd.to_datetime(date_fin)
# Fin : 2 mois plus tard
end_date = initial_date + pd.DateOffset(months=10)
# Générer les dates hebdomadaires
list_dates = pd.date_range(start=initial_date, end=end_date, freq="7D").to_list()
# Affichage
# print(list_dates)
list_dates_str = [f"'{d.strftime('%Y-%m-%d')}'" for d in list_dates]
print(list_dates_str)

["'2018-01-15'", "'2018-01-22'", "'2018-01-29'", "'2018-02-05'", "'2018-02-12'", "'2018-02-19'", "'2018-02-26'", "'2018-03-05'", "'2018-03-12'", "'2018-03-19'", "'2018-03-26'", "'2018-04-02'", "'2018-04-09'", "'2018-04-16'", "'2018-04-23'", "'2018-04-30'", "'2018-05-07'", "'2018-05-14'", "'2018-05-21'", "'2018-05-28'", "'2018-06-04'", "'2018-06-11'", "'2018-06-18'", "'2018-06-25'", "'2018-07-02'", "'2018-07-09'", "'2018-07-16'", "'2018-07-23'", "'2018-07-30'", "'2018-08-06'", "'2018-08-13'", "'2018-08-20'", "'2018-08-27'", "'2018-09-03'", "'2018-09-10'", "'2018-09-17'", "'2018-09-24'", "'2018-10-01'", "'2018-10-08'", "'2018-10-15'", "'2018-10-22'", "'2018-10-29'", "'2018-11-05'", "'2018-11-12'"]


In [7]:
len(data)

96096

In [8]:
idx

NameError: name 'idx' is not defined

In [10]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, adjusted_rand_score

ARI = []
idx = 1  # on ne commence pas à la première date car déjà fait dans le modèle initial
while idx + 1 <= len(
    list_dates_str
):  # tant que le dataframe ne contient pas tous les clients
    # extraire les clients
    date = list_dates_str[idx]

    sql_query = f"""
	WITH favourite_product AS (
		SELECT customer_id,
			   product_category_name_english AS type_produit_prefere
		FROM (
			SELECT o.customer_id,
				   p.product_category_name,
				   t.product_category_name_english,
				   count(p.product_id) AS nb_products_by_category,
				   ROW_NUMBER() OVER (PARTITION BY o.customer_id ORDER BY COUNT(p.product_id) DESC) AS rang
			FROM orders o
			JOIN order_items i ON i.order_id = o.order_id
			JOIN products p ON p.product_id = i.product_id
			JOIN translation t ON p.product_category_name = t.product_category_name
			GROUP BY o.customer_id,
					 p.product_category_name,
					 t.product_category_name_english
		)
		WHERE rang = 1
	),

	paiement_par_commande AS (
		SELECT order_id,
			   SUM(payment_value) AS depense_commande
		FROM order_pymts
		GROUP BY order_id
	),

	items_par_commande AS (
		SELECT order_id,
			   COUNT(order_item_id) AS nb_produits_commande,
			   SUM(price) AS prix_produit_commande,
			   SUM(freight_value) AS prix_transport_commande
		FROM order_items
		GROUP BY order_id
	),

	reviews_par_commande AS (
		SELECT order_id,
			   AVG(review_score) AS score_moyen_commande
		FROM order_reviews
		GROUP BY order_id
	)

	SELECT c.customer_id,
		   c.customer_state,
		   COUNT(DISTINCT o.order_id) AS nb_commandes,
		   SUM(i.nb_produits_commande) AS nb_produits,
		   SUM(i.prix_produit_commande) AS prix_produit_total,
		   SUM(i.prix_transport_commande) AS prix_transport_total,
		   SUM(p.depense_commande) AS depense_totale,
		   (julianday({date}) - julianday(MAX(o.order_purchase_timestamp))) AS recence,
		   AVG(r.score_moyen_commande) AS score_moyen,
		   f.type_produit_prefere
	FROM customers c
	JOIN orders o ON c.customer_id = o.customer_id
	LEFT JOIN items_par_commande i ON i.order_id = o.order_id
	LEFT JOIN reviews_par_commande r ON o.order_id = r.order_id
	LEFT JOIN paiement_par_commande p ON o.order_id = p.order_id
	LEFT JOIN favourite_product f ON f.customer_id = c.customer_id
	GROUP BY c.customer_unique_id
	HAVING recence>=0;
	"""

    conn = connect(database="olist.db")
    data = pd.read_sql(sql_query, conn)
    print(data.shape)

    data_filtered = data.loc[
        np.abs(
            data["depense_totale"]
            - (data["prix_produit_total"] + data["prix_transport_total"])
        )
        < (0.01 * data["depense_totale"]),
        :,
    ]
    data_filtered = data_filtered.drop(
        ["prix_transport_total", "prix_produit_total"], axis=1
    )
    data_filtered = data_filtered.dropna(inplace=False)
    
    print(f"Il y a {len(data_filtered)} clients après filtrage des erreurs")
    numeric_df = data_filtered.select_dtypes(include=numerics)

    # predictions from initial model
    predicted_labels = initial_model.predict(numeric_df)

    # predictions from new model
    new_model = Pipeline(
        steps=[
            ("scaler", PowerTransformer(method="yeo-johnson")),
            ("clusterer", KMeans(n_clusters=3, random_state=42)),
        ]
    )
    new_labels = new_model.fit_predict(numeric_df)

    # Compute ARI :
    score = adjusted_rand_score(new_labels, predicted_labels)
    print(f"{date} \nScore ARI: {score}")
    ARI.append(score)
    
    

    idx += 1


plt.plot(list_dates_str[1:], ARI)

(48109, 10)
Il y a 46106 clients après filtrage des erreurs
'2018-01-22' 
Score ARI: 0.2471695678148964
(49643, 10)
Il y a 47575 clients après filtrage des erreurs
'2018-01-29' 
Score ARI: 1.0
(51185, 10)
Il y a 49070 clients après filtrage des erreurs
'2018-02-05' 
Score ARI: 1.0
(52711, 10)
Il y a 50535 clients après filtrage des erreurs
'2018-02-12' 
Score ARI: 0.2076880276888407
(54285, 10)
Il y a 52044 clients après filtrage des erreurs
'2018-02-19' 
Score ARI: 1.0
(55953, 10)
Il y a 53665 clients après filtrage des erreurs
'2018-02-26' 
Score ARI: 0.2307967157251067
(57781, 10)
Il y a 55443 clients après filtrage des erreurs
'2018-03-05' 
Score ARI: 1.0
(59377, 10)
Il y a 56982 clients après filtrage des erreurs
'2018-03-12' 
Score ARI: 1.0
(60933, 10)
Il y a 58496 clients après filtrage des erreurs
'2018-03-19' 
Score ARI: 0.2747399946172293
(62616, 10)
Il y a 60136 clients après filtrage des erreurs
'2018-03-26' 
Score ARI: 1.0
(64044, 10)
Il y a 61532 clients après filtrage de

KeyboardInterrupt: 

Essayer sans la récence : qu'est-ce que ça change. 

In [ ]:
# Compute data drift
report = Report(metrics=[DataDriftPreset()])
report.run(reference_data=reference_data, current_data=data)
report.show()

Date initiale = 01/01/2018. Kmeans initial + power transformer sur tout ce qui précède. (customiser la requete et calculer la récence de façon propre dans la requête. ) Ajout d'une semaine de données : prédict dur kmeans préentrainé sur tous les clients. Puis réentrainement d'un nouveau modèle. 

ARI : métrique qui permet de mesurer la distance entre clusters entre les prédictions : entre modèle initial et réentrainé. 0.8: seuil ? 

Profil des clients ont changé ? Datadrift avec librairie evidently. Clients avant date déterminée et clients avant autre date --> test stat de comparaison des distributions. 